In [101]:
import datetime
import json
import uuid
import xml.etree.ElementTree as ET
from collections import OrderedDict
import os

import pandas as pd
import numpy as np
import argparse
import xmltodict
from cassandra.cluster import Cluster

In [39]:
def unix_time(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    delta = dt - epoch
    return delta.total_seconds()


def unix_time_millis(dt):
    return int(unix_time(dt) * 1000.0)


def generate_stmt(session, table, columns):
    col_exp = ', '.join(columns)
    val_exp = ', '.join(['?' for _ in range(len(columns))])
    stmt = """INSERT INTO {table} ({columns}) VALUES({values})""".format(table=table, columns=col_exp, values=val_exp)
    return session.prepare(stmt)

In [68]:
def parse_meta_data(raw_meta_data):
    tree = ET.parse(raw_meta_data)  # element tree
    root = tree.getroot()
    meta_data = OrderedDict()
    mess_cols = []
    json_data = []
    for i, child in enumerate(root):
        tag = child.tag
        if tag == 'KOPF' or tag == 'ZEIT':
            for subchild in child:
                value = subchild.text
                if subchild.tag in ['ANFANG', 'ENDE']:
                    value = unix_time_millis(datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S'))
                meta_data[subchild.tag] = value
        elif tag == 'DATEN':
            for subchild in child:
                if subchild.tag == 'SPALTE':
                    col = subchild.text
                    index = subchild.attrib.get('messstelle', '0')
                    mess_cols.append('.'.join([col, index]))
        else:
            json_data.append(xmltodict.parse(ET.tostring(child, encoding='utf-8', method='xml')))
    meta_data['json'] = json.dumps(json_data, ensure_ascii=False, separators=(';', ':')).replace(',', '.').replace(';', ',')
    return meta_data, mess_cols

In [69]:
meta, mess_cols = parse_meta_data('data/Maschine 77 (1045670479)/main.xml')

In [70]:
mess_cols

['LAENGE.1',
 'DRAHT1_V.1',
 'DURCH_MITTEL.1',
 'OVAL.1',
 'DURCH_MITTEL.2',
 'OVAL.2',
 'SPALT_Y.1',
 'SPALT_Y.2',
 'DRAHT1_T.1',
 'ZIEHST1_T.2',
 'ZIEHST1_F.3',
 'ABSPUL1_F.4',
 'STATUS_BIT.0']

In [71]:
data = pd.read_csv('data/Maschine 77 (1045670479)/mess.txt', sep='\t', header=0, names=mess_cols)

In [72]:
data.head()

,LAENGE.1,DRAHT1_V.1,DURCH_MITTEL.1,OVAL.1,DURCH_MITTEL.2,OVAL.2,SPALT_Y.1,SPALT_Y.2,DRAHT1_T.1,ZIEHST1_T.2,ZIEHST1_F.3,ABSPUL1_F.4,STATUS_BIT.0
0,"0,0","4,64","-999,00000","-999,00","-999,00000","-999,00",-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,"-999,00","-999,00","-999,00","-999,00",0006
1,"0,1","5,08","0,57362","3,90","-999,00000","-999,00",-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,"-999,00","-999,00","-999,00","-999,00",0006
2,"0,2","5,08","0,57497","1,61","-999,00000","-999,00",-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,"-999,00","-999,00","-999,00","-999,00",0006
3,"0,3","2,59","0,57875","1,32","-999,00000","-999,00",-999;-999;-999;-999;-999;-999;-999;-999;6406;-140,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,"-999,00","-999,00","-999,00","-999,00",0006
4,"0,4","5,08","0,58014","1,46","-999,00000","-999,00",-142;-135;-132;-121;-126;-122;-118;-115;-114;-111,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999,"-999,00","-999,00","-999,00","-999,00",0006


In [73]:
data.shape

(11448, 13)

In [78]:
data = data.rename(columns={'LAENGE.1': 'LAENGE'})

In [80]:
df = data.melt(id_vars=['LAENGE'], var_name='VARIABLE', value_name='VAL')

In [81]:
df[df.VAL == '-999;-999;-999;-999;-999;-999;-999;-999;-999;-999'].head()

,LAENGE,VARIABLE,VAL
57240,"0,0",SPALT_Y.1,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999
57241,"0,1",SPALT_Y.1,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999
57242,"0,2",SPALT_Y.1,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999
68596,"1135,6",SPALT_Y.1,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999
68597,"1135,7",SPALT_Y.1,-999;-999;-999;-999;-999;-999;-999;-999;-999;-999


In [82]:
%time df.LAENGE = df.LAENGE.str.replace(',', '.')
%time df.VAL = df.VAL.str.replace(',', '.')

CPU times: user 66.6 ms, sys: 2.52 ms, total: 69.1 ms
Wall time: 67.8 ms
CPU times: user 58.6 ms, sys: 370 µs, total: 59 ms
Wall time: 58.9 ms


In [83]:
def column_to_rows(df, column, sep):
    
    def _duplicate(new_row, value):
        copy = new_row.copy()
        copy[column] = value
        return copy
    
    def _cell_to_rows(row, new_rows):
        split_row = row[column].split(sep)
        new_row = row.to_dict()
        new_rows += [_duplicate(new_row, value) for value in split_row]
    new_rows = []
    df.apply(lambda row: _cell_to_rows(row, new_rows), axis=1)
    new_df = pd.DataFrame(new_rows)
    return new_df

In [84]:
%time df2 = column_to_rows(df, 'VAL', ';')

CPU times: user 4.47 s, sys: 106 ms, total: 4.58 s
Wall time: 4.53 s


In [85]:
df2.head()

,LAENGE,VAL,VARIABLE
0,0.0,4.64,DRAHT1_V.1
1,0.1,5.08,DRAHT1_V.1
2,0.2,5.08,DRAHT1_V.1
3,0.3,2.59,DRAHT1_V.1
4,0.4,5.08,DRAHT1_V.1


In [86]:
df2.dtypes

LAENGE      object
VAL         object
VARIABLE    object
dtype: object

In [87]:
%time df2.LAENGE = pd.to_numeric(df2.LAENGE, errors='coerce')
%time df2.VAL = pd.to_numeric(df2.VAL, errors='coerce')

CPU times: user 118 ms, sys: 4.85 ms, total: 122 ms
Wall time: 121 ms
CPU times: user 176 ms, sys: 2.44 ms, total: 179 ms
Wall time: 179 ms


In [88]:
df2.dtypes

LAENGE      float64
VAL         float64
VARIABLE     object
dtype: object

In [89]:
df2.shape

(343440, 3)

In [97]:
split = df2.VARIABLE.str.rsplit('.')

In [102]:
df2['VARIABLE'] = split.str.get(0)
df2['MESSSTELLE'] = split.str.get(1).astype(np.int8)

In [103]:
df2.head()

,LAENGE,VAL,VARIABLE,MESSSTELLE
0,0.0,4.64,DRAHT1_V,1
1,0.1,5.08,DRAHT1_V,1
2,0.2,5.08,DRAHT1_V,1
3,0.3,2.59,DRAHT1_V,1
4,0.4,5.08,DRAHT1_V,1


In [106]:
df2[(df2.VARIABLE=='DURCH_MITTEL') & (df2.LAENGE==10.0)]

,LAENGE,VAL,VARIABLE,MESSSTELLE
11548,10.0,0.58638,DURCH_MITTEL,1
34444,10.0,0.68032,DURCH_MITTEL,2


In [104]:
df2.dtypes

LAENGE        float64
VAL           float64
VARIABLE       object
MESSSTELLE       int8
dtype: object

In [109]:
tuple(df2.head().VARIABLE.values)

('DRAHT1_V', 'DRAHT1_V', 'DRAHT1_V', 'DRAHT1_V', 'DRAHT1_V')